In [1]:
#Utiliser une base de données

import sqlite3
conn = sqlite3.connect('imdb.db')
c = conn.cursor()
c.execute("select * from name_basics limit 10")
for row in c:
  print(row)
conn.close()

(1, 'Fred Astaire', 1899, 1987, 'soundtrack,actor,miscellaneous')
(2, 'Lauren Bacall', 1924, 2014, 'actress,soundtrack')
(3, 'Brigitte Bardot', 1934, None, 'actress,soundtrack,music_department')
(4, 'John Belushi', 1949, 1982, 'actor,soundtrack,writer')
(5, 'Ingmar Bergman', 1918, 2007, 'writer,director,actor')
(6, 'Ingrid Bergman', 1915, 1982, 'actress,soundtrack,producer')
(7, 'Humphrey Bogart', 1899, 1957, 'actor,soundtrack,producer')
(8, 'Marlon Brando', 1924, 2004, 'actor,soundtrack,director')
(9, 'Richard Burton', 1925, 1984, 'actor,soundtrack,producer')
(10, 'James Cagney', 1899, 1986, 'actor,soundtrack,director')


In [12]:
#Etapes pour afficher une table SQL

import sqlite3
conn = sqlite3.connect('imdb.db')
c = conn.cursor()
c.execute("select * from name_basics limit 10")

def tableSQL_tableau(sql):
  """
  fonction qui convertit un tableau sqlite3 en tableau python
  parametres:
             sql, un tableau sqlite3
  renvoie une liste python (dont chaque élément est une ligne de 'sql')
  """
  tableau = sql.fetchall()
  return tableau

sortie = tableSQL_tableau(c)
conn.close()

def taille_colonne(tableau):
  """
  fonction qui renvoie la taille minimale que doit avoir chaque colonne du tableau à afficher
  parametres:
             tableau, une liste de liste python
  renvoie une liste de valeur
  """
  longueur_colonne = [0] * len(tableau[0])
  for ligne in tableau:
    for column in range(len(ligne)):
      if len(str(ligne[column])) > longueur_colonne[column]:
        longueur_colonne[column] = len(str(ligne[column]))
  for i in range(len(longueur_colonne)):
    longueur_colonne[i] += 1    
  return longueur_colonne

taille = taille_colonne(sortie)

def separation(dimension):
  """
  fonction qui renvoie la representation d'un trait entre deux lignes d'un tableau
  parametres:
             dimension, une liste de valeur
  renvoie une chaine de caracteres
  """
  ligne_separation = "+"
  for column in dimension:
    ligne_separation += "-" * column
    ligne_separation += "+"
  return ligne_separation
  
ligne = separation(taille)

def afficher_ligne(ligne, dimension):
  """
  fonction qui permet d'afficher un tableau avec une ligne seulement avec des dimension de colonnes précise
  parametres:
             ligne, une liste
             dimension, une liste
  affiche la ligne au format ascii
  """
  chaine = "|"
  for column in range(len(ligne)):
    chaine += str(ligne[column])
    chaine += " " * (dimension[column] - len(str(ligne[column])))
    chaine += "|"
  print(chaine)
  
print(ligne)
afficher_ligne(sortie[0], taille)
print(ligne)

+---+----------------+-----+-----+------------------------------------+
|1  |Fred Astaire    |1899 |1987 |soundtrack,actor,miscellaneous      |
+---+----------------+-----+-----+------------------------------------+


In [11]:
#fonction finale

def afficher_table(table, debut = 0, fin = None):
  """
  fonction que affiche une table sql, avec la possibilité de choisir l'intervalle de ligne à afficher
  parametres:
             table, une table sqlite3
             debut (optionnel), un nombre entier
             fin (optionnel), un nombre entier
  affiche la partie du tableau souhaiter, par défaut son entièreté
  """
  if type(table) != list:
    sortie = tableSQL_tableau(table)
  else:
    sortie = table
  dimension_colonne = taille_colonne(sortie)
  separation_ligne = separation(dimension_colonne)
  if fin == None or fin > len(sortie):
    fin = len(sortie)
  if debut != 0:
    debut -= 1
  if debut < 1:
    debut = 0
  print(separation_ligne)
  for ligne in range(debut, fin):
    afficher_ligne(sortie[ligne], dimension_colonne)
    print(separation_ligne)  
    

import sqlite3
conn = sqlite3.connect('imdb.db')
c = conn.cursor()
c.execute("select * from name_basics limit 10")
afficher_table(c,6,8)
conn.close()

+---+----------------+-----+-----+------------------------------------+
|6  |Ingrid Bergman  |1915 |1982 |actress,soundtrack,producer         |
+---+----------------+-----+-----+------------------------------------+
|7  |Humphrey Bogart |1899 |1957 |actor,soundtrack,producer           |
+---+----------------+-----+-----+------------------------------------+
|8  |Marlon Brando   |1924 |2004 |actor,soundtrack,director           |
+---+----------------+-----+-----+------------------------------------+


In [12]:
# Pour n'avoir qu'une partie du tableau
# A améliorer, notamment sur le 'pbm' d'affichega avec des colonnes trop grande 

def projection_table(table, *args):
  """
  fonction permettent de faire une projection sur un tableau sql
  parametres:
             table, une table sqlite3
             args, une liste d'entier
  renvoie une liste avec les colonne selectionner
  """
  tableau = tableSQL_tableau(table)
  return [[ligne[column] for column in args] for ligne in tableau]
  
import sqlite3
conn = sqlite3.connect('imdb.db')
c = conn.cursor()
c.execute("select * from name_basics limit 10")
t = conn.cursor()
t.execute("select * from name_basics limit 10")
afficher_table(c, 6)
afficher_table(projection_table(t,1,4), 6) 
conn.close()

+---+----------------+-----+-----+------------------------------------+
|6  |Ingrid Bergman  |1915 |1982 |actress,soundtrack,producer         |
+---+----------------+-----+-----+------------------------------------+
|7  |Humphrey Bogart |1899 |1957 |actor,soundtrack,producer           |
+---+----------------+-----+-----+------------------------------------+
|8  |Marlon Brando   |1924 |2004 |actor,soundtrack,director           |
+---+----------------+-----+-----+------------------------------------+
|9  |Richard Burton  |1925 |1984 |actor,soundtrack,producer           |
+---+----------------+-----+-----+------------------------------------+
|10 |James Cagney    |1899 |1986 |actor,soundtrack,director           |
+---+----------------+-----+-----+------------------------------------+
+----------------+------------------------------------+
|Ingrid Bergman  |actress,soundtrack,producer         |
+----------------+------------------------------------+
|Humphrey Bogart |actor,soundtrack,produ

In [11]:
# pour melanger 2 table

def produit_cartesien(table1, table2):
  """
  fonction permettant de faire le produit cartesien de 2 tables sql
  parametres:
             table1, une table sql
             table2, une autre table sql
  renvoie une liste
  """
  tableau1 = tableSQL_tableau(table1)
  tableau2 = tableSQL_tableau(table2)
  tableau = []
  for ligne1 in tableau1:
    for ligne2 in tableau2:
      tableau += [ligne1 + ligne2]
  return tableau

import sqlite3
conn = sqlite3.connect('imdb.db')
c = conn.cursor()
c1 = conn.cursor()
c.execute("select * from title_basics limit 2")
c1.execute("select * from title_basics limit 2")
print("-----------------")
print(c.fetchall())
print("-----------------")
t = conn.cursor()
t1 = conn.cursor()
t.execute("select * from title_ratings limit 3")
t1.execute("select * from title_ratings limit 3")
afficher_table(c)
afficher_table(t)
afficher_table(produit_cartesien(c1, t1))
conn.close()

-----------------
[(3, 'short', 'Pauvre Pierrot', 'Pauvre Pierrot', 0, 1892, None, 4, 'Animation,Comedy,Romance'), (10, 'short', 'Leaving the Factory', "La sortie de l'usine Lumière à Lyon", 0, 1895, None, 1, 'Documentary,Short')]
-----------------


NameError: name 'afficher_table' is not defined

In [21]:
import sqlite3

class database:
  #https://docs.python.org/3/library/sqlite3.html
  def __init__(self, base):
    self.base = ""
    
  def connexion(self):
    """
    méthode permettant de se connecter à la base de donnée
    """
    self.con = sqlite3.connect(self.base)
    self.cur = self.con.cursor()
    
  def deconnexion(self):
    """
    méthode permettant de se deconnecter à la base de donnée
    """
    self.con.close()
    
  def fetch(self,sql):
    """
    méthode permettant d'obtenir le résultat d'une requête sql
    parametres:
               sql, une chaine de caracteres correspondant à une requete SQL
    renvoie le résultat de la requete dans la base
    """
    self.connexion()
    self.cur.execute(sql)
    result = self.cur.fetchall()
    self.deconnexion()
    return result
  
  def execute(self,sql):
    """
    méthode permettant d'executer une requete sql dans la base, plus pour les modifications de la base
    parametres:
               sql, une chaine de caracteres correspondant a une requete sql
    """
    self.connexion()
    self.cur.execute(sql)
    self.deconnexion()
    
  def chargersql():
    pass
  
  def afficher_table():
    pass
  
  def listedesrequetes():
    pass
  
  def infotable():
    pass 
  
  def informations_base():
    pass

In [22]:
import sqlite3
imdb = database("imdb.db")
imdb.connexion()
test = imdb.fetch("select primaryName from name_basics limit 10")
print(test)


OperationalError: no such table: name_basics

In [15]:
#stockage des requetes dans un dictionnaire
import os

question = []

def lire_questions(fichier):
  alire = open(fichier, "r").readlines()
  liste_questions = []
  req = False
  ligne = ""
  for l in alire:
    if l[0] == "#":
      liste_questions += [l[1:-2]]
  return liste_questions
    
  
print(lire_questions("requetes/alire.md"))

['01.Quels sont les differents types de titres dans cette base de donnees ?', '02.Combien y a-t-il de titres dans cette base de donnees ? ', '03.En quelle annee est sortie le film  The Godfather  ? ', '04.En quelle annee est sortie le premier film Superman ? ', '05.Quel est le titre original du film  Les dents de la mer  ? ', '06.Quel est le metier dâ€™Olivier Nakache ? ', '07.Quels sont les films dâ€™Olivier Nakache ? ', '08.Quel est le film ayant recueilli le plus de votes ? ', '09.Qui a ecrit le scenario du film Taxi sorti en 1998 ? ', '10.Quelles sont les noms et roles (category et job) des personnes intervenant dans la production du film Return of the Jedi ? ', '11.Quels sont les titres des films notes plus de 9 sur 10 avec plus de 10 000 votes ? ', '12.Quelle sont les 5 comedies romantiques les mieux notees ? ', '13.Quels sont les 10 films dâ€™animation ayant recu plus de 1000 votes les mieux notes ? ', '14.Combien de films durent plus de 3 heures ? ', '15.Quelle est la duree moy

In [ ]:
#echec
import os

def listes_requetes(fichier):
  alire = open(fichier, "r").readlines()
  lignes = [l for l in alire if l != ""]
  print(lignes)
  liste_requetes = []
  
listes

In [ ]:
def fichier_SQL(repertoire = "", extension = "sql"):
  verifier que repertoire existe
  verifier qu il n est pas vide
  charger dans un dico ==> ["repere"]: ("question", "requetes")
    
def test(dico, numero):
  execute requetes
  affiche resultat requetes
  
def sql():
  retourne listes de listes

In [3]:
# On fait en sorte de retourner une chaine de caracteres à la place des prints
def taille_colonne(tableau):
  """
  fonction qui renvoie la taille minimale que doit avoir chaque colonne du tableau à afficher
  parametres:
             tableau, une liste de liste python
  renvoie une liste de valeur
  """
  longueur_colonne = [0] * len(tableau[0])
  for ligne in tableau:
    for column in range(len(ligne)):
      if len(str(ligne[column])) > longueur_colonne[column]:
        longueur_colonne[column] = len(str(ligne[column]))
  for i in range(len(longueur_colonne)):
    longueur_colonne[i] += 1    
  return longueur_colonne

def separation(dimension):
  """
  fonction qui renvoie la representation d'un trait entre deux lignes d'un tableau
  parametres:
             dimension, une liste de valeur
  renvoie une chaine de caracteres
  """
  ligne_separation = "+"
  for column in dimension:
    ligne_separation += "-" * column
    ligne_separation += "+"
  return ligne_separation
  

def afficher_ligne(ligne, dimension):
  """
  fonction qui permet d'afficher un tableau avec une ligne seulement avec des dimension de colonnes précise
  parametres:
             ligne, une liste
             dimension, une liste
  affiche la ligne au format ascii
  """
  chaine = "|"
  for column in range(len(ligne)):
    chaine += str(ligne[column])
    chaine += " " * (dimension[column] - len(str(ligne[column])))
    chaine += "|"
  return chaine
  
def afficher_table(table, debut = 0, fin = None):
  chaine_finale = ""
  """
  fonction que affiche une table sql, avec la possibilité de choisir l'intervalle de ligne à afficher
  parametres:
             table, une table sqlite3
             debut (optionnel), un nombre entier
             fin (optionnel), un nombre entier
  affiche la partie du tableau souhaiter, par défaut son entièreté
  """
  if type(table) != list:
    sortie = tableSQL_tableau(table)
  else:
    sortie = table
  dimension_colonne = taille_colonne(sortie)
  separation_ligne = separation(dimension_colonne)
  if fin == None or fin > len(sortie):
    fin = len(sortie)
  if debut != 0:
    debut -= 1
  if debut < 1:
    debut = 0
  chaine_finale += separation_ligne + "\n"
  for ligne in range(debut, fin):
    chaine_finale += afficher_ligne(sortie[ligne], dimension_colonne) + "\n"
    chaine_finale += separation_ligne + "\n"
  return chaine_finale